# How can we conver long data format into wide data format

---

In this notebook I'll show you how you can convert long dataframe into wide dataframe with repetitive values in all column

In [1]:
import pandas as pd
import numpy as np

Let create some artificial data as dictionary and then convert if to Pandas DataFrame object

In [2]:
data = {'customer': ['Store_A']*12,
        'class': ['1A','1A','2B','2B','3C','3C']*2,
        'week':['19/08/2018','26/08/2018']*6, 
        'type':['Food']*6 + ['Beverage']*6, 
        'value': [np.nan,np.nan,1,1.5,1.1,1.2,np.nan,np.nan,0.96,0.70,0.96,0.96]}

In [3]:
df = pd.DataFrame(data)

In [4]:
df

,customer,class,week,type,value
0,Store_A,1A,19/08/2018,Food,NaN
1,Store_A,1A,26/08/2018,Food,NaN
2,Store_A,2B,19/08/2018,Food,1.00
3,Store_A,2B,26/08/2018,Food,1.50
4,Store_A,3C,19/08/2018,Food,1.10
5,Store_A,3C,26/08/2018,Food,1.20
6,Store_A,1A,19/08/2018,Beverage,NaN
7,Store_A,1A,26/08/2018,Beverage,NaN
8,Store_A,2B,19/08/2018,Beverage,0.96
9,Store_A,2B,26/08/2018,Beverage,0.70


In our ```DataFrame``` we have several columns with very few unqiue values. Our aim is to convert it into wide format. Namely, we want to move rows underneath that have same ```customer```, ```week```, and ```type``` to be on 1 single row. In other words, we want to wipe rows starting from index 6 and concatenate what's left along axis one. Our desirable format looks like the following:

In [5]:
d = {'customer': ['Store_A']*4,
    'week':['08/19/2018','08/26/2018']*2,
    'type':['Food']*2 + ['Beverage']*2,
     'class_1': ['1A','1A','1A', '1A'],
     'value_1': [np.nan, np.nan, np.nan, np.nan],
     'class_2': ['2B','2B', '2B', '2B'],
     'value_2': [1, 1.5, 0.96, 0.7],
     'class_3': ['3C','3C', '3C', '3C'],
     'value_3': [1.1, 1.2, 0.96, 0.96]}

In [6]:
pd.DataFrame(d)

,customer,week,type,class_1,value_1,class_2,value_2,class_3,value_3
0,Store_A,08/19/2018,Food,1A,NaN,2B,1.00,3C,1.10
1,Store_A,08/26/2018,Food,1A,NaN,2B,1.50,3C,1.20
2,Store_A,08/19/2018,Beverage,1A,NaN,2B,0.96,3C,0.96
3,Store_A,08/26/2018,Beverage,1A,NaN,2B,0.70,3C,0.96


So, to solve this problem and avoid duplicate index values we can use ```cumcount``` as a counter, then ```groupby``` with three columns, which are ```['customer', 'week', 'type']``` then set new index by using these three columns and groupby object and then reset index.

In [7]:
# Groupby
g = (df.groupby(['customer','week', 'type']).cumcount() + 1).astype(str)

# Set new index
new_df = df.set_index(['customer', 'week', 'type', g]).unstack().sort_index(level=1, axis=1)

# Change column names
new_df.columns = new_df.columns.map('_'.join)

# Reset multiindex
new_df.reset_index(inplace=True)

new_df

,customer,week,type,class_1,value_1,class_2,value_2,class_3,value_3
0,Store_A,19/08/2018,Beverage,1A,NaN,2B,0.96,3C,0.96
1,Store_A,19/08/2018,Food,1A,NaN,2B,1.00,3C,1.10
2,Store_A,26/08/2018,Beverage,1A,NaN,2B,0.70,3C,0.96
3,Store_A,26/08/2018,Food,1A,NaN,2B,1.50,3C,1.20
